# Weyl 群扩张群的结构

## 主要函数

1. Thetas(s,n) 返回生成元集合 \{\theta_i\}_{i=1}^n
2. OrderMatrixOfGens(gens) 返回生成元的群阶矩阵
3. GroupByOrderMathix(order_mat) 群阶矩阵作为泛性质，生成群
4. GroupTreeOfMaxDepth(G,depth) 生成群树（BFS）
5. UniversalPropertyOfGroup(G,max_depth=-1) 返回生成元的泛性质关系 

### Thetas(s,n)

In [ ]:
def ExpOfNilpotentMat(mat, max_depth=36, base_ring=ZZ):
    """
    1. 返回幂零矩阵的指数矩阵
    2. max_depth 指定 x^n=0 中，n 的最大值
    3. 默认基环为整数环
    4. 输入必须为 Sage 矩阵类型
    """
    dim = mat.dimensions()[0]
    exp_mat = matrix.identity(dim) # 单位矩阵
    new,i = mat,1
    while not new.is_zero(): # 运算直到 new 为零矩阵
        exp_mat += 1/factorial(i) * new
        i += 1
        new *= mat
    return matrix(ZZ,exp_mat)

In [ ]:
# 不能用自然表示，一定是伴随表示
def Thetas(s,n):
    """返回给定型的生成元集合 {\theta_i}_{i=1}^n"""
    exp = ExpOfNilpotentMat # 简写函数名
    L = LieAlgebra(ZZ,cartan_type=[s,n])
    # 一组基元
    basis = L.basis()
    # 上三角 e_i
    es = [L.e(i+1).adjoint_matrix() for i in range(n)]
    # 下三角 f_i
    fs = [L.f(i+1).adjoint_matrix() for i in range(n)]
    thetas = [exp(es[i])*exp(-fs[i])*exp(es[i]) for i in range(n)]
    return thetas

In [ ]:
# gap_str = lambda s:gap('"'+s+'"')
# gap.Read(gap_str("big_weyl_group.py"))

# 函数测试
for i in range(3):
    print(len(MatrixGroup(Thetas("A",i+1))))

### OrderMatrixOfGens

In [ ]:
def OrderMatrixOfGens(gens):
    """
    1. 计算 gens 的群阶矩阵
    2. 生成元的群基环必须为整数环
    """
    orders = [[(a*b).multiplicative_order() for a in gens] for b in gens]
    return matrix(ZZ,orders)

In [ ]:
# 函数测试
gens = Thetas("A",3)
print(gens[0].base_ring())
OrderMatrixOfGens(gens)

### 泛性质构造商群

In [ ]:
def GroupByOrderMatrix(order_mat):
    """群阶矩阵 -> 自由群商群"""
    n = order_mat.dimensions()[0]
    FG = FreeGroup(n) # 自由群
    gens = FG.gens() # 生成元
    # 生成关系
    relations = [(gens[i]*gens[j])^order_mat[i,j] for i in range(n) for j in range(n)]
    return FG/relations

In [ ]:
n = 2
gens = Thetas("A",n)
order_mat = OrderMatrixOfGens(gens)
print(order_mat)
G = GroupByOrderMatrix(order_mat)
G.1 * G.0 == G.1 * G.1

### 生成群树

In [ ]:
def GroupTreeOfMaxDepth(G,depth=-1):
    """
    1. BFS 获取群元素，按长度分层
    2. 获取前 depth 层，首层记0，默认获取整树（有限群）"""
    # 群树，初始为零层
    tree = [[G.one()]] 
    if depth == 0: # 只有零层
        return tree
    gens = G.gens() # 生成元
    S = [G.one()] # 已生成的群元素
    tree.append([]) # 添加一层
    # 将 gens 化简（去重复元素，以及单位元）
    for a in gens:
        if a not in S:
            S.append(a)
            tree[-1].append(a)
    if depth == 1: # 只有一层
        return tree
    gens = tree[-1] # 简化的生成元
    
    # 循环开始，直到深度足够，或者群生成完毕
    while depth-1 and len(tree[-1]):
        depth -= 1
        tree.append([])
        for a in tree[-2]:
            for b in gens:
                c = a * b
                if c not in S:
                    S.append(c)
                    tree[-1].append(c)
    return tree

In [ ]:
# 测试一
mat = matrix(CoxeterMatrix(["A",2]))
print(mat)
G = GroupByOrderMatrix(mat)
GroupTreeOfMaxDepth(G)

In [ ]:
# 测试二
n = 2
gens = Thetas("A",n)
order_mat = OrderMatrixOfGens(gens)
print(order_mat)
G2 = GroupByOrderMatrix(order_mat)
GroupTreeOfMaxDepth(G2,3)

### UniversalPropertyOfGens

In [5]:
def UniversalPropertyOfGroup(G,max_depth=-1):
    """
    1. 求群生成元的泛性质
    2. G 的生成元不能存在相等，或者等于1
    3. max_depth 获取层数，默认遍历整树
    """
    if max_depth == 0:
        return [[G.one()]],[[""]],{}
    # 检验输入
    gens = list(G.gens()) # 生成元
    S = [G.one()] + gens # 已生成群元素
    assert len(S)==len(set(S)),"G 的生成元不能存在相等，或者等于1"
    
    # 群树：元素形式和字符串形式
    tree = [[G.one()],gens] 
    tree_str = [[""],["%d"%i for i in range(len(gens))]]
    # 生成关系
    relations = {}
    while max_depth-1:
        max_depth -= 1
        new_layer = []
        new_layer_str = []
        for s,a in zip(tree_str[-1],tree[-1]):
            for si,b in enumerate(gens):
                c = a * b
                c_str = s + "-%d"%si
                if c in new_layer: # 化简到同一层
                    if not any(s in c_str for s in relations):
                        relations[c_str] = new_layer_str[new_layer.index(c)]
                    continue
                for i,layer in enumerate(tree):
                    if c in layer: # 化简到前边层
                        if not any(s in c_str for s in relations):
                            relations[c_str] = tree_str[i][layer.index(c)]
                        break
                else:
                    new_layer.append(c)
                    new_layer_str.append(c_str)
        if len(new_layer):
            tree.append(new_layer)
            tree_str.append(new_layer_str)
        else:
            break
    return tree,tree_str,relations

### 测试

In [12]:
# 导入模块
load("weyl_group_extension.sage")

In [8]:
from pprint import pprint
def show_tree(tree):
    """打印群树"""
    for i in tree_str:
        for j in i:
            print(j,end="\t")
        print()
    return

def relations2elements(relations,gens):
    """将生成元转为群元素"""
    one = gens[0]/gens[0]
    elements = []
    for key in relations:
        new = one
        for i in key.split("-"):
            new *= gens[int(i)]
        if not relations[key]: # 指向单位元
            elements.append(new)
            continue
        for i in relations[key][::-1].split("-"):
            new /= gens[int(i)]
        elements.append(new)
    return elements

#### 测试一

In [6]:
# 测试一，检验有限型 Coxeter 群
mat = matrix(CoxeterMatrix(["A",3]))
print(mat)
G = GroupByOrderMatrix(mat)
tree1 = GroupTreeOfMaxDepth(G)
tree,tree_str,relations = UniversalPropertyOfGroup(G)
print(tree1[:-1] == tree)
relations

[1 3 2]
[3 1 3]
[2 3 1]
True


{'0-0': '',
 '1-1': '',
 '2-0': '0-2',
 '2-2': '',
 '1-0-1': '0-1-0',
 '2-1-2': '1-2-1',
 '2-1-0-2': '1-2-1-0'}

In [ ]:
# 打印群树
show_tree(tree_str)

#### 测试二

In [14]:
# 测试二，检验无限群
# 初始化
s,n = "A",3
gens = Thetas(s,n)
order_mat = OrderMatrixOfGens(gens)
print("群阶矩阵（泛性质）",order_mat, sep="\n")
G = GroupByOrderMatrix(order_mat)

# 计算结果
k = 4 # 取前 k 层
tree1 = GroupTreeOfMaxDepth(G,k)
tree,tree_str,relations = UniversalPropertyOfGroup(G,k)
print(tree1 == tree)
relations

[2 3 2]
[3 2 3]
[2 3 2]


KeyboardInterrupt: 

#### 测试三

In [ ]:
# 测试三，I型 Coxeter 群，仅当全部遍历时，泛性质才完整
mat = matrix([[1,20],[20,1]])
print(mat)
G = GroupByOrderMatrix(mat)
tree1 = GroupTreeOfMaxDepth(G,10)
tree,tree_str,relations = UniversalPropertyOfGroup(G,9)
print(tree1 == tree)
relations

#### 测试四

In [ ]:
# 测试四，补充泛性质后的规律
for n in range(5):
    mat = matrix(CoxeterMatrix(["A",n+1]))
    print(mat)
    G = GroupByOrderMatrix(mat)
    _,tree_str,relations = UniversalPropertyOfGroup(G)
    print({key:relations[key] for key in relations.keys() if len(set(list(key)))>2})

#### 测试五

In [ ]:
# 测试五，用前 k 阶泛性质生成商群 -->> 局部泛性质 √
n,k = 2,4
gens = Thetas("A",n)
order_mat = OrderMatrixOfGens(gens)
print("泛性质矩阵",order_mat,sep="\n")
G = GroupByOrderMatrix(order_mat)
tree,tree_str,relations = UniversalPropertyOfGroup(G,k)
print("前%d层导出关系"%(k+1),relations, sep="\n")
# 定义商群
FG = FreeGroup(2)
elements = relations2elements(relations,FG.gens())
print("商集",elements,sep="\n")
newG = FG/elements
[len(i) for i in GroupTreeOfMaxDepth(newG,k+1)]==[len(i) for i in GroupTreeOfMaxDepth(G,k+1)]

In [ ]:
[len(i) for i in GroupTreeOfMaxDepth(newG,13)]==[len(i) for i in GroupTreeOfMaxDepth(G,13)]

In [ ]:
_,_,relations = UniversalPropertyOfGroup(G,13)
relations
# {'0000': '',
#  '1111': '',
#  '01010': '111',
#  '10101': '000',
#  '000111': '1010',
#  '111000': '0101',
#  '10100101': '00011000',
#  '11100111': '01011010',
#  '0001100011000': '10100100101'}

#### 测试六

In [ ]:
# 测试六，weyl扩张群的泛性质
s,n = "A",4
gens = Thetas(s,n)
G = MatrixGroup(gens)
gens = G.gens()

In [ ]:
k = 3
_,_,relations = UniversalPropertyOfGroup(G,k)
print("前%d层导出关系"%(k+1),relations, sep="\n")
# 定义商群
FG = FreeGroup(n)
elements = relations2elements(relations,FG.gens())
print("商集",elements,sep="\n")
newG = FG/elements
[len(i) for i in GroupTreeOfMaxDepth(newG,k+1)]==[len(i) for i in GroupTreeOfMaxDepth(G,k+1)]